In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator 

from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [ ]:
#load the image you'd like to make a prediction for
image = load_img(image_path, target_size=(150,150))
image = img_to_array(image)
image = image/255 #convert to grayscale
image = np.expand_dims(image, axis=0)

#build the VGG16 network
model = VGGFace(model='resnet50')
#model = ResNet50(include_top=False, weights='imagenet')

#get the bottleneck prediction from the pre-trained VGG16 model
bottleneck_prediction = model.predict(image)

#build top model
model = Sequential()
model.add(Flatten(input_shape=bottleneck_prediction.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5,activation='sigmoid'))

#load the weights you saved after training the top layers
model.load_weights('top_model_weights.h5')

#plug the bottleneck prediction into the top model to get the final classification
class_predicted = model.predict_classes(bottleneck_prediction)

#show the result!
result = class_predicted[0]
print(result)

In [ ]:
# from https://towardsdatascience.com/boost-your-cnn-image-
#     classifier-performance-with-progressive-resizing-in-keras-a7d96da06e20

prior = keras.applications.VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(48, 48, 3)
)
model = Sequential()
model.add(prior)
model.add(Flatten())
model.add(Dense(256, activation='relu', name='Dense_Intermediate'))
model.add(Dropout(0.1, name='Dropout_Regularization'))
model.add(Dense(12, activation='sigmoid', name='Output'))


# Freeze the VGG16 model, e.g. do not train any of its weights.
# We will just use it as-is.
for cnn_block_layer in model.layers[0].layers:
    cnn_block_layer.trainable = False
model.layers[0].trainable = False


# Compile the model. I found that RMSprop with the default learning
# weight worked fine.
model.compile(
    optimizer=RMSprop(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#
# Step 4: fit the model.
#
# Finally we fit the model. I use two callbacks here: EarlyStopping,
# which stops the model short of its full 20 epochs if validation 
# performance consistently gets worse; and ReduceLROnPlateau, which 
# reduces the learning rate 10x at a time when it detects model 
# performance is no longer improving between epochs.
#

# Recall that our dataset is highly imbalanced. We deal with this
# problem by generating class weights and passing them to the model
# at training time. The model will use the class weights to adjust
# how it trains so that each class is considered equally important to
# get right, even if the actual distribution of images is highly 
# variable.
import os
labels_count = dict()
for img_class in [ic for ic in os.listdir('images_cropped/') if ic[0] != '.']:
    labels_count[img_class] = len(os.listdir('images_cropped/' + img_class))
total_count = sum(labels_count.values())
class_weights = {cls: total_count / count for cls, count in 
                 enumerate(labels_count.values())}


model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batch_size,
    class_weight=class_weights,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),
        ReduceLROnPlateau(patience=2)
    ]
)